## 只使用时间diff, 调整XgbOOST学习率=0.005，n_estimators=5000

In [33]:
import pandas as pd
import pickle
with open('./X_data.pkl','rb') as file:
    X_data=pickle.load(file)
with open('./Y_data.pkl','rb') as file:
    Y_data=pickle.load(file)
with open('./X_test.pkl','rb') as file:
    X_test=pickle.load(file)

In [34]:
X_data[['regDate','creatDate']]

,regDate,creatDate
0,20040402,20160404
1,20030301,20160309
2,20040403,20160402
3,19960908,20160312
4,20120103,20160313
...,...,...
149995,20000607,20160327
149996,20091102,20160312
149997,20101003,20160328
149998,20060312,20160401


In [35]:
# 对日期格式进行清洗
def f(x):
    x=str(x)
    if x[4:6]=='00':
        return x[0:4]+'-'+'01'+'-'+x[6:8]
    else:
        return x[0:4]+'-'+x[4:6]+'-'+x[6:8]
    
X_data['regDate']=pd.to_datetime(X_data['regDate'].apply(f)) #这里apply运用了一个自己写的函数，来调整时间格式
X_data['creatDate']=pd.to_datetime(X_data['creatDate'].apply(f))
X_data[['regDate','creatDate']]        

,regDate,creatDate
0,2004-04-02,2016-04-04
1,2003-03-01,2016-03-09
2,2004-04-03,2016-04-02
3,1996-09-08,2016-03-12
4,2012-01-03,2016-03-13
...,...,...
149995,2000-06-07,2016-03-27
149996,2009-11-02,2016-03-12
149997,2010-10-03,2016-03-28
149998,2006-03-12,2016-04-01


In [36]:
# 对X_test数据也要处理一下
X_test['regDate']=pd.to_datetime(X_test['regDate'].apply(f)) #这里apply运用了一个自己写的函数，来调整时间格式
X_test['creatDate']=pd.to_datetime(X_test['creatDate'].apply(f))

In [37]:
#提取时间diff
#1991-01-01
X_data['regDate'].min()
X_test['regDate'].min()
base_date=pd.to_datetime('1991-01-01')
X_data['regDate_diff']=(X_data['regDate']-base_date).dt.days #间隔的天数
X_test['regDate_diff']=(X_test['regDate']-base_date).dt.days

In [38]:
#提取时间diff
#1991-01-01
X_data['creatDate'].min()
X_test['creatDate'].min()
base_date=pd.to_datetime('2014-03-10')# 这里应该找训练集，测试集中最小的数据

X_data['creatDate_diff']=(X_data['creatDate']-base_date).dt.days #间隔的天数
X_test['creatDate_diff']=(X_test['creatDate']-base_date).dt.days


In [39]:
#构造新特征'creatDate-regDate
X_data['creatDate-regDate']=(X_data['creatDate']-X_data['regDate']).dt.days #注意（）括号
X_test['creatDate-regDate']=(X_test['creatDate']-X_test['regDate']).dt.days #注意（）括号

In [40]:
#去掉原始的日期类型
X_data.drop(['regDate','creatDate'],axis=1,inplace=True)
X_test.drop(['regDate','creatDate'],axis=1,inplace=True)


In [41]:
## 数据集切分
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(X_data,Y_data,test_size=0.2,random_state=2021)
import lightgbm as lgb
#创建xgb模型
lgb_model=lgb.LGBMRegressor(
        num_leaves=2**5-1,reg_alpha=0.25,reg_lambda=0.25,objective='regression',
        max_depth=1,learning_rate=0.005,min_child_samples=3, random_state=2021,
        n_estimators=6000, subsample=1,colsample_bytree=1)
lgb_model.fit(x_train,y_train)
y_pred=lgb_model.predict(x_val)

In [42]:
import numpy as np
#统计price的分布
def show_stats(data):
    print('min',np.min(data))
    print('max',np.max(data))
    print('ptp',np.ptp(data)) #计算最大值与最小值的差(极差) peak to peak
    print('mean',np.mean(data))
    print('std',np.std(data)) #标准差
    print('median',np.median(data))
#查询price的统计
show_stats(y_pred)

min -3333.836413470717
max 71979.27727670276
ptp 75313.11369017349
mean 5917.955961789614
std 7015.707534018446
median 3323.358116023834


In [47]:
y_pred

array([1193.64696132, 2035.56683028, 8919.48770719, ..., 6008.84716278,
       5506.51443182, 4751.4529417 ])

In [44]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_pred,y_val)

1082.1283107025315

In [45]:
#重新训练模型,使用全量数据
#创建xgb模型
lgb_model.fit(X_data,Y_data)
y_pred=lgb_model.predict(X_test)
show_stats(y_pred)

min -3657.1350616548384
max 71286.62264008349
ptp 74943.75770173833
mean 5918.34481250645
std 7073.469016929869
median 3249.490268353302


In [52]:
#读取xgb_baseline3.csv
xgb_pred=pd.read_csv('./xgb_baseline3.csv')
xgb_pred
xgb_pred['lgb_price']=y_pred
xgb_pred
xgb_pred.loc[xgb_pred['lgb_price']<11,'lgb_price']=11
#xgb效果好，设置xgb权重为0.7，lightgbm权重为0.3
xgb_pred['price']=xgb_pred['price']*0.7+xgb_pred['lgb_price']*0.3
xgb_pred

,SaleID,price,lgb_price
0,200000,1273.541278,1193.646961
1,200001,1892.347299,2035.566830
2,200002,8690.269412,8919.487707
3,200003,1144.895865,952.681849
4,200004,2011.647156,2065.075952
...,...,...,...
49995,249995,6460.905046,6642.866888
49996,249996,19155.738938,19774.566792
49997,249997,5937.279749,6008.847163
49998,249998,5272.193680,5506.514432


In [56]:
#输出结果
result=xgb_pred[['SaleID','price']]
result.to_csv('./xgb_lgb_baseline1.csv',index=False)

### 使用XGBoost,对日期特征进行了处理
# Score=585.2867